In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import omegaconf
from shell.utils.experiment_utils import *
from shell.fleet.utils.fleet_utils import *
from shell.utils.metric import *
import matplotlib.pyplot as plt
from shell.fleet.network import TopologyGenerator
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
from shell.fleet.fleet import Agent, Fleet
from shell.fleet.data.data_utilize import *
from shell.fleet.data.recv import *

from sklearn.manifold import TSNE
from torchvision.utils import make_grid
from shell.fleet.data.data_utilize import *
import logging
from sklearn.metrics import f1_score
import os
from shell.fleet.data.recv_utils import *
from tqdm import tqdm
import argparse
from functools import partial
from torchvision.utils import make_grid
from shell.utils.oodloss import OODSeparationLoss
from pythresh.thresholds.dsn import DSN
from pythresh.thresholds.aucp import AUCP
from pythresh.thresholds.boot import BOOT
from pythresh.thresholds.zscore import ZSCORE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import defaultdict

logging.basicConfig(level=logging.INFO)

In [3]:
dataset = "mnist"
algo = "modular"

experiment_folder = "experiment_results"
experiment_name = "jorge_setting_lowest_task_id_wins_modmod_test_sync_base_True_opt_with_random_False_frozen_False_transfer_decoder_True_transfer_structure_True_no_sparse_basis_True"

use_contrastive = False
num_trains_per_class = 64
seed = 0
num_tasks = 10
parallel = False
comm_freq = 2 

ranker = 'instance'
# ranker = 'label'

module_select = 'tryout'
# module_select = 'trust_sim'

save_dir = get_save_dir(experiment_folder, experiment_name,
                        dataset, algo, num_trains_per_class, use_contrastive, seed)


In [4]:
def modify_cfg(net_cfg, agent_cfg, train_cfg, fleet_additional_cfg, cfg):
    cfg.sharing_strategy['ranker'] = ranker
    cfg.sharing_strategy['module_select'] = module_select
    cfg.sharing_strategy['num_shared_module'] = 2
    cfg.sharing_strategy['num_tryout_epochs'] = 1
    # cfg.sharing_strategy['sync_base'] = False
    # del fleet_additional_cfg['fake_dataset']
    return net_cfg, agent_cfg, train_cfg, fleet_additional_cfg, cfg

In [8]:
fleet = setup_fleet(save_dir, task_id=4, parallel=False,
                    modify_cfg=modify_cfg)

INFO:root:Class sequence: [5 0 4 3 4 6 4 3 8 0 1 0 5 9 6 4 9 0 3 0]
INFO:root:task 0 :(128, 1, 28, 28) (100, 1, 28, 28) (1872, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28) (100, 1, 28, 28) (1992, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28) (100, 1, 28, 28) (1940, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28) (100, 1, 28, 28) (1992, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28) (100, 1, 28, 28) (1954, 1, 28, 28)


{'train': {'component_update_freq': 100, 'num_epochs': 100, 'init_component_update_freq': 100, 'init_num_epochs': 100, 'save_freq': 10}, 'dataset': {'dataset_name': 'mnist', 'num_tasks': 10, 'num_classes_per_task': 2, 'with_replacement': True, 'num_trains_per_class': 64, 'num_vals_per_class': 50, 'remap_labels': True}, 'net': {'name': 'mlp', 'depth': 4, 'layer_size': 64, 'dropout': 0.5, 'no_sparse_basis': True}, 'sharing_strategy': {'name': 'modmod', 'num_coms_per_round': 2, 'module_selection': 'gt_most_similar', 'pre_or_post_comm': 'pre', 'opt_with_random': False, 'comm_freq': 100, 'sync_base': True, 'freeze_candidate_module': False, 'transfer_decoder': True, 'transfer_structure': True}, 'seed': 0, 'algo': 'modular', 'job_name': 'mnist_modular_numtrain_64', 'num_agents': 8, 'root_save_dir': 'experiment_results/jorge_setting_lowest_task_id_wins_modmod_test_sync_base_True_opt_with_random_False_frozen_False_transfer_decoder_True_transfer_structure_True_no_sparse_basis_True', 'parallel': 

INFO:root:task 5 :(128, 1, 28, 28) (100, 1, 28, 28) (2115, 1, 28, 28)
INFO:root:task 6 :(128, 1, 28, 28) (100, 1, 28, 28) (1901, 1, 28, 28)
INFO:root:task 7 :(128, 1, 28, 28) (100, 1, 28, 28) (1940, 1, 28, 28)
INFO:root:task 8 :(128, 1, 28, 28) (100, 1, 28, 28) (1989, 1, 28, 28)
INFO:root:task 9 :(128, 1, 28, 28) (100, 1, 28, 28) (1990, 1, 28, 28)
INFO:root:Class sequence: [4 7 5 7 6 0 3 0 5 0 3 6 2 7 6 7 6 1 0 5]
INFO:root:task 0 :(128, 1, 28, 28) (100, 1, 28, 28) (2010, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28) (100, 1, 28, 28) (1920, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28) (100, 1, 28, 28) (1938, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28) (100, 1, 28, 28) (1990, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28) (100, 1, 28, 28) (1872, 1, 28, 28)
INFO:root:task 5 :(128, 1, 28, 28) (100, 1, 28, 28) (1968, 1, 28, 28)
INFO:root:task 6 :(128, 1, 28, 28) (100, 1, 28, 28) (2060, 1, 28, 28)
INFO:root:task 7 :(128, 1, 28, 28) (100, 1, 28, 28) (1986, 1, 28, 28)
INFO:root:task 8 :(128

i_size 28
num_classes 2
net_cfg {'name': 'mlp', 'depth': 4, 'layer_size': 64, 'dropout': 0.5, 'no_sparse_basis': True, 'i_size': 28, 'num_classes': 2, 'num_tasks': 10, 'num_init_tasks': 4, 'use_contrastive': False}
<class 'shell.learners.er_dynamic.CompositionalDynamicER'>


INFO:root:task 7 :(128, 1, 28, 28) (100, 1, 28, 28) (2010, 1, 28, 28)
INFO:root:task 8 :(128, 1, 28, 28) (100, 1, 28, 28) (1989, 1, 28, 28)
INFO:root:task 9 :(128, 1, 28, 28) (100, 1, 28, 28) (2145, 1, 28, 28)
INFO:root:Agent: node_id: 69420, seed: 69420000
INFO:root:Agent: node_id: 0, seed: 0
INFO:root:Agent: node_id: 1, seed: 1000
INFO:root:Agent: node_id: 2, seed: 2000
INFO:root:Agent: node_id: 3, seed: 3000
INFO:root:Agent: node_id: 4, seed: 4000
INFO:root:Agent: node_id: 5, seed: 5000
INFO:root:Agent: node_id: 6, seed: 6000
INFO:root:Agent: node_id: 7, seed: 7000
INFO:root:Created fleet with 8 agents
INFO:root:Adding neighbors...
INFO:root:Fleet initialized


Setting seed 69420000
Setting seed 0
Setting seed 1000
Setting seed 2000
Setting seed 3000
Setting seed 4000
Setting seed 5000
Setting seed 6000
Setting seed 7000


In [9]:
fleet.eval_test(task_id=5)

[{0: 0.9905378486055777,
  1: 0.9488752556237219,
  2: 0.9198588709677419,
  3: 0.9593858345715701,
  4: 0.9263050153531218,
  5: 0.5366430260047281,
  'avg': 0.8802676418544103},
 {0: 0.9895418326693227,
  1: 0.9452965235173824,
  2: 0.9198588709677419,
  3: 0.9593858345715701,
  4: 0.655448717948718,
  5: 0.5132113821138211,
  'avg': 0.8304571936314261},
 {0: 0.9895418326693227,
  1: 0.9468302658486708,
  2: 0.9198588709677419,
  3: 0.9583952451708767,
  4: 0.7424749163879598,
  5: 0.49381443298969074,
  'avg': 0.8418192606723771},
 {0: 0.9880478087649402,
  1: 0.9493865030674846,
  2: 0.9173387096774194,
  3: 0.9574046557701833,
  4: 0.8831376734258272,
  5: 0.49975235264982665,
  'avg': 0.8658446172259469},
 {0: 0.9890438247011952,
  1: 0.9488752556237219,
  2: 0.9188508064516129,
  3: 0.9598811292719168,
  4: 0.95523765574527,
  5: 0.5176233635448136,
  'avg': 0.8815853392230882},
 {0: 0.9895418326693227,
  1: 0.9493865030674846,
  2: 0.9173387096774194,
  3: 0.9598811292719168,
 

In [10]:
fleet.change_save_dir('task_similarity_engineering_experiment_results')

In [11]:
fleet.communicate(task_id=5, end_epoch=None, comm_freq=None, num_epochs=None)

INFO:root:epochs: 0, training task: 5
INFO:root:	task: 0	loss: 0.03743382	acc: 0.991
INFO:root:	task: 1	loss: 0.22928971	acc: 0.949
INFO:root:	task: 2	loss: 0.33064962	acc: 0.920
INFO:root:	task: 3	loss: 0.20791444	acc: 0.959
INFO:root:	task: 4	loss: 0.27937189	acc: 0.926
INFO:root:	task: 5	loss: 0.40082524	acc: 0.983
INFO:root:	task: avg	loss: 0.24758079	acc: 0.955


no. current components 5 NUM_CANDIDATE_MODULES 1 len(module_list) 1


INFO:root:epochs: 1, training task: 5
INFO:root:	task: 0	loss: 0.03743382	acc: 0.991
INFO:root:	task: 1	loss: 0.22928971	acc: 0.949
INFO:root:	task: 2	loss: 0.33064962	acc: 0.920
INFO:root:	task: 3	loss: 0.20791444	acc: 0.959
INFO:root:	task: 4	loss: 0.27937189	acc: 0.926
INFO:root:	task: 5	loss: 0.52011716	acc: 0.986
INFO:root:	task: avg	loss: 0.26746277	acc: 0.955
INFO:root:candidate 5: W/update: 0.97, WO/update: 0.95, improv 0.02105263157894739
INFO:root:Not keeping any new modules. Total: 5
INFO:root:epochs: 2, training task: 5
INFO:root:	task: 0	loss: 0.03743382	acc: 0.991
INFO:root:	task: 1	loss: 0.22928971	acc: 0.949
INFO:root:	task: 2	loss: 0.33064962	acc: 0.920
INFO:root:	task: 3	loss: 0.20791444	acc: 0.959
INFO:root:	task: 4	loss: 0.27937189	acc: 0.926
INFO:root:	task: 5	loss: 0.38979909	acc: 0.987
INFO:root:	task: avg	loss: 0.24574310	acc: 0.955
INFO:root:final components: 5
INFO:root:epochs: 0, training task: 5
INFO:root:	task: 0	loss: 0.03743382	acc: 0.991
INFO:root:	task:

no. current components 5 NUM_CANDIDATE_MODULES 1 len(module_list) 1


INFO:root:epochs: 1, training task: 5
INFO:root:	task: 0	loss: 0.03743382	acc: 0.991
INFO:root:	task: 1	loss: 0.22928971	acc: 0.949
INFO:root:	task: 2	loss: 0.33064962	acc: 0.920
INFO:root:	task: 3	loss: 0.20791444	acc: 0.959
INFO:root:	task: 4	loss: 0.27937189	acc: 0.926
INFO:root:	task: 5	loss: 0.68785165	acc: 0.463
INFO:root:	task: avg	loss: 0.29541852	acc: 0.868
INFO:root:candidate 5: W/update: 0.46, WO/update: 0.46, improv 0.0
INFO:root:Not keeping any new modules. Total: 5
INFO:root:epochs: 2, training task: 5
INFO:root:	task: 0	loss: 0.03743382	acc: 0.991
INFO:root:	task: 1	loss: 0.22928971	acc: 0.949
INFO:root:	task: 2	loss: 0.33064962	acc: 0.920
INFO:root:	task: 3	loss: 0.20791444	acc: 0.959
INFO:root:	task: 4	loss: 0.27937189	acc: 0.926
INFO:root:	task: 5	loss: 0.67938246	acc: 0.463
INFO:root:	task: avg	loss: 0.29400699	acc: 0.868
INFO:root:final components: 5
INFO:root:epochs: 0, training task: 5
INFO:root:	task: 0	loss: 0.03743382	acc: 0.991
INFO:root:	task: 1	loss: 0.22928

...

no. current components 5 NUM_CANDIDATE_MODULES 1 len(module_list) 1


INFO:root:epochs: 1, training task: 5
INFO:root:	task: 0	loss: 0.03555028	acc: 0.988
INFO:root:	task: 1	loss: 0.26426935	acc: 0.946
INFO:root:	task: 2	loss: 0.33472061	acc: 0.920
INFO:root:	task: 3	loss: 0.20777596	acc: 0.960
INFO:root:	task: 4	loss: 0.27535241	acc: 0.886
INFO:root:	task: 5	loss: 0.68978746	acc: 0.501
INFO:root:	task: avg	loss: 0.30124268	acc: 0.867
INFO:root:candidate 5: W/update: 0.6, WO/update: 0.6, improv 0.0
INFO:root:Not keeping any new modules. Total: 5
INFO:root:epochs: 2, training task: 5
INFO:root:	task: 0	loss: 0.03555028	acc: 0.988
INFO:root:	task: 1	loss: 0.26426935	acc: 0.946
INFO:root:	task: 2	loss: 0.33472061	acc: 0.920
INFO:root:	task: 3	loss: 0.20777596	acc: 0.960
INFO:root:	task: 4	loss: 0.27535241	acc: 0.886
INFO:root:	task: 5	loss: 0.68480521	acc: 0.501
INFO:root:	task: avg	loss: 0.30041230	acc: 0.867
INFO:root:final components: 5
INFO:root:epochs: 0, training task: 5
INFO:root:	task: 0	loss: 0.03555028	acc: 0.988
INFO:root:	task: 1	loss: 0.2642693

no. current components 5 NUM_CANDIDATE_MODULES 1 len(module_list) 1


INFO:root:epochs: 1, training task: 5
INFO:root:	task: 0	loss: 0.03555028	acc: 0.988
INFO:root:	task: 1	loss: 0.26426935	acc: 0.946
INFO:root:	task: 2	loss: 0.33472061	acc: 0.920
INFO:root:	task: 3	loss: 0.20777596	acc: 0.960
INFO:root:	task: 4	loss: 0.27535241	acc: 0.886
INFO:root:	task: 5	loss: 0.69430210	acc: 0.499
INFO:root:	task: avg	loss: 0.30199512	acc: 0.867
INFO:root:candidate 5: W/update: 0.4, WO/update: 0.4, improv 0.0
INFO:root:Not keeping any new modules. Total: 5
INFO:root:epochs: 2, training task: 5
INFO:root:	task: 0	loss: 0.03555028	acc: 0.988
INFO:root:	task: 1	loss: 0.26426935	acc: 0.946
INFO:root:	task: 2	loss: 0.33472061	acc: 0.920
INFO:root:	task: 3	loss: 0.20777596	acc: 0.960
INFO:root:	task: 4	loss: 0.27535241	acc: 0.886
INFO:root:	task: 5	loss: 0.69892891	acc: 0.499
INFO:root:	task: avg	loss: 0.30276625	acc: 0.867
INFO:root:final components: 5


perfs [0.8669661614840246, 0.9350891388303029, 0.8669661614840246, 0.8666425368885554]


In [12]:
node = fleet.agents[0]
node.modmod_record.df

,task_id,source_task_id,task_sim,module_id,neighbor_id,tryout_module_perf
0,5,4,0.333333,4,4,NaN
1,6,4,0.333333,4,5,NaN
2,7,4,1.000000,4,6,NaN
3,8,4,0.333333,4,5,NaN
4,9,4,0.333333,4,7,NaN
0,5,4,-0.219018,4,4,0.955366


In [14]:
node.train_kwargs

{'component_update_freq': 100,
 'num_epochs': 100,
 'init_component_update_freq': 100,
 'init_num_epochs': 100,
 'save_freq': 10,
 'module_list': [Linear(in_features=64, out_features=64, bias=True)],
 'decoder_list': [{'decoder': Linear(in_features=64, out_features=2, bias=True),
   'source_class_labels': array([2, 1])}],
 'structure_list': [{'structure': Parameter containing:
   tensor([[1.4816, 0.4443, 0.4523, 0.6124],
           [0.4903, 1.4981, 1.5775, 1.4950],
           [0.5579, 0.4729, 0.4973, 0.9931],
           [0.8211, 0.4625, 0.6263, 0.7021],
           [1.2320, 0.8768, 0.7979, 0.9631]], device='cuda:0', requires_grad=True),
   'module_id': 4}]}

In [17]:
node.modmod_record.path

'task_similarity_engineering_experiment_results/agent_0/modmod_add_modules_record.csv'

In [13]:
node.save_dir

'task_similarity_engineering_experiment_results/agent_0'

In [ ]:
node.incoming_modules

In [ ]:
task_sims = [-0.6105588176717575, -0.6862172730042327, -0.693143989000467, -0.6848385105159178, -0.6034522980633152]

most_similar_task = max(
            range(len(task_sims)), key=lambda x: (task_sims[x], -x))
most_similar_task

In [ ]:
fleet.agents[1].outgoing_modules

In [ ]:
len(node.dataset.valset[0])

In [ ]:
neighbor_id = 1
query = node.query_tasks[neighbor_id]
fig, ax = plt.subplots(1, len(query.keys()), figsize=(10, 10))
for i, (y, data) in enumerate(query.items()):
    X = data['X'].cpu()
    print(data['scores'])
    ax[i].imshow(make_grid(X).permute(1, 2, 0));

In [ ]:
from shell.fleet.mod.modmod import compute_consistency
compute_consistency(node.net, X.to('cuda'), task_id=0)

In [ ]:
node.eval_test(4)

In [ ]:
with torch.no_grad():
    node.net.eval()
    for i, (y, data) in enumerate(query.items()):
        X = data['X']
        for t in range(4):
            logits = node.net(X, t)
            print('t:', t, 'y:', y, '\nlogits', logits)
    

In [ ]:
node = fleet.agents[5]
X = node.dataset.valset[3].tensors[0]
# X = node.dataset.valset[6].tensors[0]
plt.imshow(make_grid(X).permute(1, 2, 0));

In [ ]:
print(fleet.agents[0].dataset.class_sequence)
print(fleet.agents[1].dataset.class_sequence)

In [ ]:
task_id = 3
task = node.dataset.class_sequence[task_id *
                                                 node.dataset.num_classes_per_task: (task_id + 1) * node.dataset.num_classes_per_task]
task

In [ ]:
node.dataset.class_sequence

In [ ]:
node.incoming_modules

In [ ]:
fleet.agents[1].query_tasks

In [ ]:
import torch

In [ ]:
dist = torch.tensor(
    [[1, 0.5, 0.2],
     [0.2, 0.01, -0.1]]
)

In [ ]:
dist, _ = torch.topk(dist, 2, largest=False)

In [ ]:
dist

In [ ]:
from torchmetrics.functional.pairwise import pairwise_cosine_similarity

In [ ]:
x = torch.tensor([[2, 3], [3, 5], [5, 8]], dtype=torch.float32)

In [ ]:
pairwise_cosine_similarity(x, x)